### Bag Of Words (BoW)
: 단어들의 순서를 고려하지 않고, 단어들의 빈도수에만 집중하는 텍스트 데이터의 수치화 표현 방법
1. 각 단어에 고유한 인덱스 부여 (단어 집합 생성)
2. 각 인덱스의 위치에 따라 단어 토큰의 빈도수를 기록한 벡터 생성

In [1]:
from konlpy.tag import Okt

# 한글 문장 처리를 위해
okt = Okt()

# 형태소 확인 
output = okt.morphs("\
누가 내 가슴에다 불을 질렀나 \
누가 내 심장에다 못을 박았나 \
그대의 눈빛은 날 얼어붙게 해 \
그대여 다시 내게 마음을 주오 \
") 

print(output)


['누가', '내', '가슴', '에다', '불', '을', '질렀나', '누가', '내', '심장', '에다', '못', '을', '박았나', '그대', '의', '눈빛', '은', '날', '얼어붙게', '해', '그대', '여', '다시', '내게', '마음', '을', '주오']


In [2]:
#bow함수 정의
def build_bag_of_words(input_text):
    
    input_text = input_text.replace('.','') #온점 제거
    tokenized_txt = okt.morphs(input_text) #형태소 분석
    
    word_to_idx = {}
    bow = []

    for word in tokenized_txt:
        if word not in word_to_idx.keys():
            #처음 나온 단어
            word_to_idx[word] = len(word_to_idx)
            bow.insert(len(word_to_idx) -1, 1)
        else:
            idx = word_to_idx.get(word) #키 값으로 인덱스 호출
            bow[idx] = bow[idx] + 1 #반복됐으므로 빈도수증가

    return word_to_idx, bow

In [3]:
# 한글 문장 예시 결과 확인

doc1 = "\
누가 내 가슴에다 불을 질렀나 \
누가 내 심장에다 못을 박았나 \
그대의 눈빛은 날 얼어붙게 해 \
그대여 다시 내게 마음을 주오 \
"
vocab, bow = build_bag_of_words(doc1)
print('vocabulary: ', vocab)
print('bow vector: ', bow)

vocabulary:  {'누가': 0, '내': 1, '가슴': 2, '에다': 3, '불': 4, '을': 5, '질렀나': 6, '심장': 7, '못': 8, '박았나': 9, '그대': 10, '의': 11, '눈빛': 12, '은': 13, '날': 14, '얼어붙게': 15, '해': 16, '여': 17, '다시': 18, '내게': 19, '마음': 20, '주오': 21}
bow vector:  [2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [4]:
# 사이킷런의 클래스를 이용한 BoW 확인
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

input_txt = ["Every day, every night getting wasted \
But I miss you, what did I do?\
Fuck it up, laugh it off, and I lost you"]


vector = CountVectorizer()
m = vector.fit_transform(input_txt)

# 각 단어 인덱스 확인
print('vocabulary: ', vector.vocabulary_)
# 각 단어 빈도수 확인 
print("BoW vector: ", m.toarray())

vocabulary:  {'every': 5, 'day': 2, 'night': 12, 'getting': 7, 'wasted': 15, 'but': 1, 'miss': 11, 'you': 17, 'what': 16, 'did': 3, 'do': 4, 'fuck': 6, 'it': 8, 'up': 14, 'laugh': 9, 'off': 13, 'and': 0, 'lost': 10}
BoW vector:  [[1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 2]]


### TF-IDF (Term Frequency-Inverse Document Frequency)

: DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

In [12]:
import pandas as pd
from math import log

docs = [
    '사과 맛있어',
    '사과 바나나',
    '바나나 길어 노란 바나나',
    '과일 중 제일 맛있어'
]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
vocab

['과일', '길어', '노란', '맛있어', '바나나', '사과', '제일', '중']

In [13]:
n = len(docs) #총 문서수

# tf, idf, tf-idf 함수 정의
def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(n/(df + 1))

def tfidf(t, d):
    return tf(t,d) * idf(t)

In [19]:
# tf 출력
result = []

# 각 문서에 대해 반복
for i in range(n):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일,길어,노란,맛있어,바나나,사과,제일,중
0,0,0,0,1,0,1,0,0
1,0,0,0,0,1,1,0,0
2,0,1,1,0,2,0,0,0
3,1,0,0,1,0,0,1,1


In [20]:
# idf 값 확인
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
과일,0.693147
길어,0.693147
노란,0.693147
맛있어,0.287682
바나나,0.287682
사과,0.287682
제일,0.693147
중,0.693147


In [21]:
# tf-idf 출력
result = []
for i in range(n):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns= vocab)
tfidf_

,과일,길어,노란,맛있어,바나나,사과,제일,중
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.287682,0.000000,0.000000,0.693147,0.693147


In [27]:
# 사이킷런을 이용한 DTM & TF-IDF 실습
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["every day, every night", "getting wasted", \
"but I miss you", "what did I do?"]

# DTM 확인
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) #각 단어의 빈도수 출력
print(vector.vocabulary_) #인덱스 출력

[[0 1 0 0 2 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 0]
 [1 0 0 0 0 0 1 0 0 0 1]
 [0 0 1 1 0 0 0 0 0 1 0]]
{'every': 4, 'day': 1, 'night': 7, 'getting': 5, 'wasted': 8, 'but': 0, 'miss': 6, 'you': 10, 'what': 9, 'did': 2, 'do': 3}


In [30]:
# 사이킷런 내의 메소드를 이용하여 tf-idf 값 출력
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())

[[0.         0.40824829 0.         0.         0.81649658 0.
  0.         0.40824829 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.70710678
  0.         0.         0.70710678 0.         0.        ]
 [0.57735027 0.         0.         0.         0.         0.
  0.57735027 0.         0.         0.         0.57735027]
 [0.         0.         0.57735027 0.57735027 0.         0.
  0.         0.         0.         0.57735027 0.        ]]
